In [1]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install lxml

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import requests
from bs4 import BeautifulSoup

/Users/admin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, "lxml")  # html.parser

In [6]:
table = soup.find("table", class_="wikitable")

In [7]:
tbody = table.find("tbody")

In [8]:
gdp = []

for i, tr in enumerate(tbody.find_all("tr")):
    row_data = {}
    for j, td in enumerate(tr.find_all("td")):
        if j == 0:  # 첫 번째 td에서 국가 이름 추출
            a_tag = td.find("a")
            if a_tag:
                row_data["Country"] = a_tag.text.strip()
            else:
                row_data["Country"] = td.text.strip()

        if j == 1 and td.text.strip() != "—":
            row_data["GDP_USD_billion"] = round(
                int(td.text.strip().replace(",", "")) / 1000, 2
            )

        if j == 2 and len(row_data) == 2:
            if len(td.text.strip()) > 4:
                row_data["Year"] = td.text.strip()[-4:]
            elif len(td.text.strip()) == 4:
                row_data["Year"] = td.text.strip()
    if len(row_data) > 1:
        gdp.append(row_data)

for g in gdp:
    g

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from etl_project_gdp import *


def make_df():
    log("extract start")
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # HTML에서 테이블을 DataFrame으로 읽어오기
    table = soup.find("table", class_="wikitable")
    df = pd.read_html(StringIO(str(table)))[0]  # 첫 번째 테이블만 가져오기

    # 컬럼 이름 설정
    df.columns = [
        "Country",
        "IMF_Forecast",
        "IMF_Year",
        "World_Bank_Estimate",
        "World_Bank_Year",
        "UN_Estimate",
        "UN_Year",
    ]
    df = df[["Country", "IMF_Forecast"]]
    log("extract end")
    return df


# 데이터 전처리 함수 정의
def preprocess_gdp(row):
    data = {}

    # 국가명 처리
    if pd.notna(row["Country"]):
        if row["Country"] != "World":
            data["Country"] = row["Country"]

    # gdp 처리
    if pd.notna(row["IMF_Forecast"]) and row["IMF_Forecast"] != "—":
        data["GDP_USD_billion"] = round(
            int(row["IMF_Forecast"].replace(",", "")) / 1000, 2
        )

    if len(data) > 1:
        return data


def transform_data(preprocess_gdp, df):
    log("transform start")
    gdp_data = df.apply(preprocess_gdp, axis=1).tolist()
    gdp_data = [entry for entry in gdp_data if entry is not None]
    log("transform end")
    return gdp_data


# 크롤링 후, 데이터 전처리 적용
df = make_df()
gdp_data = transform_data(preprocess_gdp, df)


# 결과 출력
# for data in gdp_data:
#     if data:
#         print(data)

/Users/admin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


 - GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.

In [10]:
gdp_over_100B_print(gdp_data)

United States  -  28781.08
China  -  18532.63
Germany  -  4591.1
Japan  -  4110.45
India  -  3937.01
United Kingdom  -  3495.26
France  -  3130.01
Brazil  -  2331.39
Italy  -  2328.03
Canada  -  2242.18
Russia  -  2056.84
Mexico  -  2017.03
Australia  -  1790.35
South Korea  -  1760.95
Spain  -  1647.11
Indonesia  -  1475.69
Netherlands  -  1142.51
Turkey  -  1113.56
Saudi Arabia  -  1106.02
Switzerland  -  938.46
Poland  -  844.62
Taiwan  -  802.96
Belgium  -  655.19
Sweden  -  623.05
Argentina  -  604.26
Ireland  -  564.02
Thailand  -  548.89
Austria  -  540.89
Israel  -  530.66
United Arab Emirates  -  527.8
Norway  -  526.95
Singapore  -  525.23
Philippines  -  471.52
Vietnam  -  465.81
Iran  -  464.18
Bangladesh  -  455.16
Malaysia  -  445.52
Denmark  -  409.99
Hong Kong  -  406.77
Colombia  -  386.08
South Africa  -  373.23
Romania  -  369.97
Egypt  -  347.59
Pakistan  -  338.24
Chile  -  333.76
Czech Republic  -  325.88
Finland  -  308.06
Portugal  -  298.95
Kazakhstan  -  296.7

 - 추출 (Extract)한 정보는 'Countries_by_GDP.json'라는 이름의 JSON 화일 포맷으로 저장해야 합니다.

In [11]:
gdp_to_json(gdp_data)

- 추출한 데이터를 데이터베이스에 저장하세요. 'Countries_by_GDP'라는 테이블명으로 'World_Economies.db'라는 데이터 베이스에 저장되어야 합니다. 
- 해당 테이블은 'Country', 'GDP_USD_billion'라는 어트리뷰트를 반드시 가져야 합니다.

In [13]:
from etl_project_gdp_with_sql import *

create_table()
delete_table()
insert_data_to_table(gdp_data)
data = select_all_data()
for d in data:
    print(d)

(1, 'United States', 28781.08)
(2, 'China', 18532.63)
(3, 'Germany', 4591.1)
(4, 'Japan', 4110.45)
(5, 'India', 3937.01)
(6, 'United Kingdom', 3495.26)
(7, 'France', 3130.01)
(8, 'Brazil', 2331.39)
(9, 'Italy', 2328.03)
(10, 'Canada', 2242.18)
(11, 'Russia', 2056.84)
(12, 'Mexico', 2017.03)
(13, 'Australia', 1790.35)
(14, 'South Korea', 1760.95)
(15, 'Spain', 1647.11)
(16, 'Indonesia', 1475.69)
(17, 'Netherlands', 1142.51)
(18, 'Turkey', 1113.56)
(19, 'Saudi Arabia', 1106.02)
(20, 'Switzerland', 938.46)
(21, 'Poland', 844.62)
(22, 'Taiwan', 802.96)
(23, 'Belgium', 655.19)
(24, 'Sweden', 623.05)
(25, 'Argentina', 604.26)
(26, 'Ireland', 564.02)
(27, 'Thailand', 548.89)
(28, 'Austria', 540.89)
(29, 'Israel', 530.66)
(30, 'United Arab Emirates', 527.8)
(31, 'Norway', 526.95)
(32, 'Singapore', 525.23)
(33, 'Philippines', 471.52)
(34, 'Vietnam', 465.81)
(35, 'Iran', 464.18)
(36, 'Bangladesh', 455.16)
(37, 'Malaysia', 445.52)
(38, 'Denmark', 409.99)
(39, 'Hong Kong', 406.77)
(40, 'Colombia',

In [14]:
from etl_project_gdp_with_sql import select_gdp_over_100B


over_100B_data = select_gdp_over_100B()
for data in over_100B_data:
    print(data)

(1, 'United States', 28781.08)
(2, 'China', 18532.63)
(3, 'Germany', 4591.1)
(4, 'Japan', 4110.45)
(5, 'India', 3937.01)
(6, 'United Kingdom', 3495.26)
(7, 'France', 3130.01)
(8, 'Brazil', 2331.39)
(9, 'Italy', 2328.03)
(10, 'Canada', 2242.18)
(11, 'Russia', 2056.84)
(12, 'Mexico', 2017.03)
(13, 'Australia', 1790.35)
(14, 'South Korea', 1760.95)
(15, 'Spain', 1647.11)
(16, 'Indonesia', 1475.69)
(17, 'Netherlands', 1142.51)
(18, 'Turkey', 1113.56)
(19, 'Saudi Arabia', 1106.02)
(20, 'Switzerland', 938.46)
(21, 'Poland', 844.62)
(22, 'Taiwan', 802.96)
(23, 'Belgium', 655.19)
(24, 'Sweden', 623.05)
(25, 'Argentina', 604.26)
(26, 'Ireland', 564.02)
(27, 'Thailand', 548.89)
(28, 'Austria', 540.89)
(29, 'Israel', 530.66)
(30, 'United Arab Emirates', 527.8)
(31, 'Norway', 526.95)
(32, 'Singapore', 525.23)
(33, 'Philippines', 471.52)
(34, 'Vietnam', 465.81)
(35, 'Iran', 464.18)
(36, 'Bangladesh', 455.16)
(37, 'Malaysia', 445.52)
(38, 'Denmark', 409.99)
(39, 'Hong Kong', 406.77)
(40, 'Colombia',

In [2]:
def countries_by_continent():
    url = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_by_continent"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # HTML에서 테이블을 DataFrame으로 읽어오기
    table = soup.find_all("table", {"class": "sortable"})
    africa_rows = table[0].find_all("tr")
    asia_rows = table[1].find_all("tr")
    europe_rows = table[2].find_all("tr")
    north_america_rows = table[3].find_all("tr")
    oceania_rows = table[4].find_all("tr")
    south_america_rows = table[5].find_all("tr")

    africa = each_continent(africa_rows)
    asia = each_continent(asia_rows)
    europe = each_continent(europe_rows)
    north_america = each_continent(north_america_rows)
    oceania = each_continent(oceania_rows)
    south_america = each_continent(south_america_rows)

    return africa, asia, europe, north_america, oceania, south_america


def each_continent(rows):
    data = []
    for row in rows[3:]:
        cols = row.find_all("td")
        if cols[0].find("a"):
            country = cols[0].find("a")
            country = country.text.strip()
            data.append(country)
    return data


africa, asia, europe, north_america, oceania, south_america = countries_by_continent()

In [3]:
for data in gdp_data:
    if data["Country"] in africa:
        data["Region"] = "Africa"
    elif data["Country"] in asia:
        data["Region"] = "Asia"
    elif data["Country"] in europe:
        data["Region"] = "Europe"
    elif data["Country"] in north_america:
        data["Region"] = "North America"
    elif data["Country"] in oceania:
        data["Region"] = "Oceania"
    elif data["Country"] in south_america:
        data["Region"] = "South America"

In [4]:
for data in gdp_data:
    if "Region" not in data:
        if data["Country"] == "South Korea":
            data["Region"] = "Asia"
        elif data["Country"] == "DR Congo":
            data["Region"] = "Africa"
        elif data["Country"] == "Congo":
            data["Region"] = "Africa"
        elif data["Country"] == "Bahamas":
            data["Region"] = "North America"
        elif data["Country"] == "Gambia":
            data["Region"] = "Africa"

In [5]:
df = pd.DataFrame(gdp_data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          194 non-null    object 
 1   GDP_USD_billion  194 non-null    float64
 2   Region           194 non-null    object 
dtypes: float64(1), object(2)
memory usage: 4.7+ KB


In [6]:
top5_per_region = (
    df.groupby("Region")
    .apply(lambda x: x.nlargest(5, "GDP_USD_billion"))
    .reset_index(drop=True)
)  # reset_index -> 인덱스 재정의, drop=True면 기존 인덱스는 삭제
top5_per_region

/var/folders/14/2vn0vwzj29zb97jgpbhnrr_00000gn/T/ipykernel_23818/2141019078.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("Region")


,Country,GDP_USD_billion,Region
0,South Africa,373.23,Africa
1,Egypt,347.59,Africa
2,Algeria,266.78,Africa
3,Nigeria,252.74,Africa
4,Ethiopia,205.13,Africa
5,China,18532.63,Asia
6,Japan,4110.45,Asia
7,India,3937.01,Asia
8,Russia,2056.84,Asia
9,South Korea,1760.95,Asia


In [7]:
average_gdp_per_region = (
    top5_per_region.groupby("Region")["GDP_USD_billion"].mean().reset_index()
)
average_gdp_per_region

,Region,GDP_USD_billion
0,Africa,289.094
1,Asia,6079.576
2,Europe,3038.302
3,North America,6657.082
4,Oceania,417.440
5,South America,787.590


In [134]:
from etl_project_gdp_with_sql import alert_new_column_region

alert_new_column_region()

In [8]:
from etl_project_gdp_with_sql import insert_region_data_to_table, delete_table

delete_table()
insert_region_data_to_table(df)

In [8]:
from etl_project_gdp_with_sql import select_gdp_top5_each_region

select_gdp_top5_each_region()

[('Africa', 289.09399999999994),
 ('Asia', 6079.576000000001),
 ('Europe', 3038.3020000000006),
 ('North America', 6657.082),
 ('Oceania', 417.43999999999994),
 ('South America', 787.5899999999999)]